# 2D Isostatic gravity inversion - True Model

Este [IPython Notebook](http://ipython.org/videos.html#the-ipython-notebook) utiliza a biblioteca de código aberto [Fatiando a Terra](http://fatiando.org/)

In [1]:
#%matplotlib inline
import numpy as np
from scipy.misc import derivative
import scipy as spy
from scipy import interpolate
import matplotlib
matplotlib.use('TkAgg', force=True)
import matplotlib.pyplot as plt
import math
import cPickle as pickle
import datetime
import string as st

from scipy.misc import imread

from __future__ import division

from fatiando import gravmag, mesher, utils, gridder
from fatiando.mesher import Prism, Polygon
from fatiando.gravmag import prism
from fatiando.utils import ang2vec, si2nt, contaminate
from fatiando.gridder import regular, profile
from fatiando.vis import mpl

from numpy.testing import assert_almost_equal
from numpy.testing import assert_array_almost_equal
from pytest import raises

plt.rc('font', size=16)

/Users/macelabastos/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
import functions as fc

## Observation coordinates.

In [3]:
# Model`s limits
ymin = 0.0
ymax = 380000.0
zmin = -1000.0
zmax = 40000.0
xmin = -100000.0
xmax = 100000.0

area = [ymin, ymax, zmax, zmin]

In [4]:
ny = 150 # number of observation datas and number of prisms along the profile
# coordinates defining the horizontal boundaries of the
# adjacent columns along the profile
y = np.linspace(ymin, ymax, ny)

In [5]:
# coordinates of the center of the columns forming the
# interpretation model
n = ny - 1
dy = (ymax - ymin)/n
ycmin = ymin + 0.5*dy
ycmax = ymax - 0.5*dy
yc = np.reshape(np.linspace(ycmin, ycmax, n),(n,1))
x = np.zeros_like(yc)
z = np.zeros_like(yc)-150.0

In [6]:
## Edge extension (observation coordinates)
sigma = 2.0
edge = sigma*dy*n

## Model parameters

In [7]:
# Model densities
# Indices and polygons relationship:
# cc = continental crust layer
# oc = ocean crust layer
# w = water layer
# s = sediment layer
# m = mantle layer
dw = np.array([1030.0])
ds0 = np.array([2600.0])
ds1 = np.array([2600.0])
dcc = np.array([2850.0])
doc = np.array([2885.0])
dm = np.array([3250.0])
#dc = dcc
# coordinate defining the horizontal boundaries of the continent-ocean boundary
COT = 165000.0
# list defining crust density variance
dc = np.zeros_like(yc)
aux = yc <= COT
for i in range(len(yc[aux])):
    dc[i] = dcc
for i in range(len(yc[aux]),n):
    dc[i] = doc    
# defining sediments layers density matrix
ds = np.vstack((np.reshape(np.repeat(ds0,n),(1,n)),np.reshape(np.repeat(ds1,n),(1,n))))

# S0 => isostatic compensation surface (Airy's model)
# SR = S0+dS0 => reference Moho (Forward modeling)
S0 = np.array([48000.0]) #original
dS0 = np.array([5000.0]) #original

## For display

## Water bottom

In [8]:
bathymetry = np.reshape(np.loadtxt('../data/etopo1-pelotas.txt'),(n,1))
tw = 0.0 - bathymetry

# Sediments surfaces

In [9]:
tos_picks = np.array([[    766.12903226,     773.55747388],
       [  76229.83870968,    1368.17355747],
       [  84657.25806452,    2359.20036347],
       [ 103427.41935484,    4341.25397547],
       [ 236733.87096774,    5332.28078146],
       [ 378084.67741935,    5926.89686506]])

In [10]:
# change the coordinates of the extremum points in order to
# avoid problems for constructing the interpolator
tos_picks[0,0] = ymin
tos_picks[-1,0] = ymax

In [11]:
tos = fc.surface_interpolate_function(tos_picks,yc)
for i in range(len(tos)):
    if tos[i] < tw[i]:
        tos[i] = tw[i]

In [12]:
#np.savetxt('../data/B-model-rifted-margin-true-tos-surface.txt', tos, fmt='%.18f')

In [13]:
# layer sediments thickness
ts0 = tos - tw

In [14]:
basement_picks = np.array([[    766.12903226,    1962.78964107],
       [   5745.96774194,    1764.58427987],
       [  37157.25806452,    2160.99500227],
       [  68185.48387097,    3350.22716947],
       [  73548.38709677,    3746.63789187],
       [  79677.41935484,    4935.87005906],
       [  85040.32258065,    8107.15583826],
       [ 100362.90322581,   21981.53112222],
       [ 104576.61290323,   24756.40617901],
       [ 111471.77419355,   28522.3080418 ],
       [ 113387.09677419,   29116.9241254 ],
       [ 116834.67741935,   29116.9241254 ],
       [ 119133.06451613,   28522.3080418 ],
       [ 122197.58064516,   27531.2812358 ],
       [ 125645.16129032,   27531.2812358 ],
       [ 132540.32258065,   28522.3080418 ],
       [ 136754.03225806,   28522.3080418 ],
       [ 145564.51612903,   26936.6651522 ],
       [ 152459.67741935,   22972.55792821],
       [ 158588.70967742,   15837.16492503],
       [ 163568.5483871 ,    8899.97728305],
       [ 167016.12903226,    7116.12903226],
       [ 173528.22580645,    6323.30758746],
       [ 222560.48387097,    6323.30758746],
       [ 227923.38709677,    7314.33439346],
       [ 232137.09677419,    9692.79872785],
       [ 237116.93548387,   13062.28986824],
       [ 242096.77419355,   15044.34348024],
       [ 252056.4516129 ,   15638.95956383],
       [ 270826.61290323,   14251.52203544],
       [ 278870.96774194,   13260.49522944],
       [ 378084.67741935,   13062.28986824]])

In [15]:
# change the coordinates of the extremum points in order to
# avoid problems for constructing the interpolator
basement_picks[0,0] = ymin
basement_picks[-1,0] = ymax

In [16]:
basement = fc.surface_interpolate_function(basement_picks,yc)
for i in range(len(basement)):
    if basement[i] < tos[i]:
        basement[i] = tos[i]

In [17]:
#np.savetxt('../data/B-model-rifted-margin-true-basement-surface.txt', basement, fmt='%.18f')

In [18]:
# layer igneous thickness
ts1 = basement - tos

In [19]:
# thickness sediments vector
ts = np.vstack((np.reshape(ts0,(1,n)),np.reshape(ts1,(1,n))))

## Moho surface

In [20]:
#picks do modelo vulcanico
moho_picks = np.array([[   1518.33556011,   37872.3690103 ],
       [   9985.48495019,   37967.53246753],
       [  18981.83117715,   37586.8786386 ],
       [  34857.73628355,   36349.75369458],
       [  56554.80659562,   34160.99417824],
       [  75076.69588642,   32543.21540528],
       [  88306.61680842,   31686.74429019],
       [ 104182.52191482,   30830.2731751 ],
       [ 118470.83651057,   29402.82131661],
       [ 127996.37957441,   27309.2252575 ],
       [ 138580.31631201,   24073.6677116 ],
       [ 148105.85937585,   23313.92745186],
       [ 192558.39367376,   23122.03313927],
       [ 207375.9051064 ,   23026.86968204],
       [ 213726.26714896,   22360.72548142],
       [ 221135.02286527,   21028.43708016],
       [ 226426.99123407,   20267.1294223 ],
       [ 231189.76276599,   20267.1294223 ],
       [ 236481.73113479,   20647.78325123],
       [ 242832.09317735,   21409.09090909],
       [ 249182.45521991,   21409.09090909],
       [ 253945.22675183,   21789.74473802],
       [ 259237.19512063,   22265.56202418],
       [ 271408.72236887,   22265.56202418],
       [ 275642.2970639 ,   21789.74473802],
       [ 278817.47808518,   21028.43708016],
       [ 283580.2496171 ,   20933.27362293],
       [ 289401.41482278,   19791.31213614],
       [ 295751.77686534,   18459.02373489],
       [ 299456.1547235 ,   18268.69682042],
       [ 304748.1230923 ,   18839.67756382],
       [ 312686.0756455 ,   19600.98522167],
       [ 321682.42187246,   19886.47559337],
       [ 332795.55544693,   19791.31213614],
       [ 337029.13014197,   19600.98522167],
       [ 342321.09851077,   18363.86027765],
       [ 345496.27953205,   17412.22570533],
       [ 380423.27076613,   17031.5718764 ]])

In [21]:
# change the coordinates of the extremum points in order to
# avoid problems for constructing the interpolator
moho_picks[0,0] = ymin
moho_picks[-1,0] = ymax

In [22]:
moho = fc.surface_interpolate_function(moho_picks,yc) + 10000.0
for i in range(len(moho)):
    if moho[i] < basement[i]:
        moho[i] = basement[i]

In [23]:
# layer mantle thickness
tm = S0 - moho

# layer crust thickness
toc = moho - tw - ts0 - ts1

In [24]:
#np.savetxt('../data/B-model-rifted-margin-true-moho-surface.txt', moho, fmt='%.18f')

## Known depths (basement)

In [ ]:
print '[', np.reshape(yc,(n,))[5], ',' , np.reshape(basement,(n,))[5], '],'
print '[', np.reshape(yc,(n,))[22], ',' , np.reshape(basement,(n,))[22], '],'
print '[', np.reshape(yc,(n,))[78], ',' , np.reshape(basement,(n,))[78], '],'
print '[', np.reshape(yc,(n,))[112], ',' , np.reshape(basement,(n,))[112], '],'

In [25]:
# Basement surface known values
basement_known = np.array([[ 14026.8456376 , 1869.0890218 ]#,
#[ 57382.5503356 , 2936.17848552 ],
#[ 200201.342282 , 6323.30758746 ],
#[ 286912.751678 , 13244.7339327 ]
                          ])

np.savetxt('../data/B-model-rifted-margin-basement-known-depths-1pts.txt', basement_known, fmt='%.18f')

In [ ]:
print '[', np.reshape(yc,(n,))[5], ',' , np.reshape(basement,(n,))[5], '],'
print '[', np.reshape(yc,(n,))[22], ',' , np.reshape(basement,(n,))[22], '],'
print '[', np.reshape(yc,(n,))[40], ',' , np.reshape(basement,(n,))[40], '],'
print '[', np.reshape(yc,(n,))[58], ',' , np.reshape(basement,(n,))[58], '],'
print '[', np.reshape(yc,(n,))[78], ',' , np.reshape(basement,(n,))[78], '],'
print '[', np.reshape(yc,(n,))[112], ',' , np.reshape(basement,(n,))[112], '],'

In [ ]:
# Basement surface known values
basement_known_more = np.array([[ 14026.8456376 , 1869.0890218 ],
[ 57382.5503356 , 2936.17848552 ],
[ 103288.590604 , 23908.1984893 ],
[ 149194.630872 , 24849.6705958 ],
[ 200201.342282 , 6323.30758746 ],
[ 286912.751678 , 13244.7339327 ]])

#np.savetxt('../data/B-model-rifted-margin-basement-more-known-depths.txt', basement_known_more, fmt='%.18f')
#np.savetxt('../data/B-model-rifted-margin-basement-known-depths-6pts.txt', basement_known_more, fmt='%.18f')

#Basement surface known values
basement_known_new = np.array([
[ 103288.590604 , 23908.1984893 ],
[ 149194.630872 , 24849.6705958 ]])

#np.savetxt('../data/B-model-rifted-margin-basement-new-known-depths.txt', basement_known_new, fmt='%.18f')

## Known depths (moho)

In [ ]:
print '[', np.reshape(yc,(n,))[22], ',' , np.reshape(moho,(n,))[22], '],'
print '[', np.reshape(yc,(n,))[78], ',' , np.reshape(moho,(n,))[78], '],'
print '[', np.reshape(yc,(n,))[112], ',' , np.reshape(moho,(n,))[112], ']'

In [ ]:
# Moho surface known values
moho_known = np.array([[ 57382.5503356 , 44088.6956005 ]
#[ 200201.342282 , 33072.9473386 ],
#[ 286912.751678 , 30279.5232356 ]
                      ])

np.savetxt('../data/B-model-rifted-margin-moho-known-depths-1pts.txt', moho_known, fmt='%.18f')

## Synthetic data

In [ ]:
# parameters vector
psyn = []
psyn = np.vstack((ts1, tm, dS0))

In [ ]:
# prisms calculation by Fatiando a Terra
prism_w_syn = fc.prism_w_function(xmax,xmin,dy,edge,dw,dcc,tw,yc)
prism_s_syn = fc.prism_s_function(xmax,xmin,dy,edge,ds,dcc,tw,psyn,yc,ts0,two_layers=True)
prism_c_syn = fc.prism_c_function(xmax,xmin,dy,edge,S0,dcc,dc,tw,psyn,yc,ts0,two_layers=True)
prism_m_syn = fc.prism_m_function(xmax,xmin,dy,edge,S0,dcc,dm,psyn,yc)

In [ ]:
# z component of gravity calculation by Fatiando a Terra
gzw_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_w_syn)
gzs0_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_s_syn[1])
gzs1_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_s_syn[2])
gzc_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_c_syn)
gzm_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_m_syn)

#Observed data calculation:
gsyn = gzw_syn + gzs0_syn + gzs1_syn + gzc_syn + gzm_syn
#gsyn = fc.g_function(x,yc,z,gzw_syn,prism_s_syn,prism_c_syn,prism_m_syn)

In [ ]:
#np.savetxt('../data/B-model-rifted-margin-synthetic-gravity-data.txt', gsyn, fmt='%.18f')

In [ ]:
sgm = 9.81*(10**(-6))*(dw*tw + ds0*ts0 + ds1*ts1 + dc*(S0-tw-ts0-ts1-tm)+dm*tm)

## Model plot

In [ ]:
polygons_water = []
for (yi, twi) in zip(yc, tw):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_water.append(Polygon(np.array([[y1, y2, y2, y1], 
                                            [0.0, 0.0, twi, twi]]).T,
                                               props={'density': dw - dcc}))
polygons_sediments0 = []
for (yi, twi, s0i) in zip(yc, np.reshape(tw,(n,)), np.reshape(tos,(n,))):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_sediments0.append(Polygon(np.array([[y1, y2, y2, y1], 
                                                [twi, twi, s0i, s0i]]).T,
                                                    props={'density': ds0 - dcc}))
    
polygons_sediments1 = []
for (yi, s0i, s1i) in zip(yc, np.reshape(tos,(n,)), np.reshape(basement,(n,))):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_sediments1.append(Polygon(np.array([[y1, y2, y2, y1], 
                                                [s0i, s0i, s1i, s1i]]).T,
                                                   props={'density': ds1 - dcc}))
    
polygons_crust = []
for (yi, si, Si, dci) in zip(yc, np.reshape(basement,(n,)), np.reshape(moho,(n,)), dc):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_crust.append(Polygon(np.array([[y1, y2, y2, y1], 
                                            [si, si, Si, Si]]).T,
                                                props={'density': dci - dcc})) 

polygons_mantle = []
for (yi, Si) in zip(yc, np.reshape(moho,(n,))):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_mantle.append(Polygon(np.array([[y1, y2, y2, y1], 
                                                [Si, Si, S0+dS0, S0+dS0]]).T,
                                                    props={'density': dm - dcc}))

In [ ]:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(12,13))

import matplotlib.gridspec as gridspec
heights = [8, 8, 8, 1]
gs = gridspec.GridSpec(4, 1, height_ratios=heights)
ax3 = plt.subplot(gs[0])
ax6 = plt.subplot(gs[1])
ax4 = plt.subplot(gs[2])
ax5 = plt.subplot(gs[3])

ax3.axhline(y=0.0, xmin=ymin, xmax=ymax, color='k', linestyle='--', linewidth=1)
#ax3.plot(0.001*yc, gobs, 'or', mfc='none', markersize=8, label='observed data')
ax3.plot(0.001*yc, gsyn, '-g', linewidth=2, label='simulated data')
ax3.set_xlim(0.001*ymin, 0.001*ymax)
ax3.set_ylabel('gravity disturbance (mGal)', fontsize=16)
ax3.set_xticklabels(['%g'% (l) for l in ax3.get_xticks()], fontsize=14)
ax3.set_yticklabels(['%g'% (l) for l in ax3.get_yticks()], fontsize=14)
ax3.legend(loc='best', fontsize=14, facecolor='silver')

ax6.plot(0.001*yc, sgm, '-g', linewidth=2, label='lithostatic stress')
ax6.set_xlim(0.001*ymin, 0.001*ymax)
ax6.set_ylim(1290,1380)
ax6.set_ylabel('Lithostatic Stress (MPa)', fontsize=16)
ax6.set_xticklabels(['%g'% (l) for l in ax6.get_xticks()], fontsize=14)
ax6.set_yticklabels(['%g'% (l) for l in ax6.get_yticks()], fontsize=14)
ax6.legend(loc='best', fontsize=14, facecolor='silver')

ax4.axhline(y=0.0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=1)
aux = yc <= COT
for (pwi) in (polygons_water):
    tmpx = [x for x in pwi.x]
    tmpx.append(pwi.x[0])
    tmpy = [y for y in pwi.y]
    tmpy.append(pwi.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='lightskyblue')
for (ps0i) in (polygons_sediments0):
    tmpx = [x for x in ps0i.x]
    tmpx.append(ps0i.x[0])
    tmpy = [y for y in ps0i.y]
    tmpy.append(ps0i.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='tan')
for (ps1i) in (polygons_sediments1):
    tmpx = [x for x in ps1i.x]
    tmpx.append(ps1i.x[0])
    tmpy = [y for y in ps1i.y]
    tmpy.append(ps1i.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='tan')
for (pci) in (polygons_crust[:len(yc[aux])]):
    tmpx = [x for x in pci.x]
    tmpx.append(pci.x[0])
    tmpy = [y for y in pci.y]
    tmpy.append(pci.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='orange')
for (pcoi) in (polygons_crust[len(yc[aux]):n]):
    tmpx = [x for x in pcoi.x]
    tmpx.append(pcoi.x[0])
    tmpy = [y for y in pcoi.y]
    tmpy.append(pcoi.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='olive')
for (pmi) in (polygons_mantle):
    tmpx = [x for x in pmi.x]
    tmpx.append(pmi.x[0])
    tmpy = [y for y in pmi.y]
    tmpy.append(pmi.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='pink')

#ax4.axhline(y=S0, xmin=ymin, xmax=ymax, color='w', linestyle='--', linewidth=3)
#ax4.axvline(x=COT, ymin=ybottom, ymax=ytop, linestyle=':', color='w', linewidth=3, label='COT')
ax4.plot(yc, tw, '-k', linewidth=3)
#ax4.plot(yc, tos, '-k', linewidth=3)
ax4.plot(yc, basement, '-k', linewidth=3, label='true surfaces')
ax4.plot(yc, moho, '-k', linewidth=3)
ax4.axhline(y=S0+dS0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=3)
#ax4.set_ylim((S0+dS0), zmin)
ax4.set_ylim((57500.0), zmin)
ax4.set_xlim(ymin, ymax)
ax4.set_xlabel('y (km)', fontsize=16)
ax4.set_ylabel('z (km)', fontsize=16)
ax4.set_xticklabels(['%g'% (0.001*l) for l in ax4.get_xticks()], fontsize=14)
ax4.set_yticklabels(['%g'% (0.001*l) for l in ax4.get_yticks()], fontsize=14)
#ax4.legend(loc='lower right', fontsize=14, facecolor='silver')    

X, Y = fig.get_dpi()*fig.get_size_inches()
plt.title('Density (kg/m$^{3}$)', fontsize=18)
ax5.axis('off')

layers_list1 = ['water', 'sediment', 'continental', 'oceanic', 'mantle']
layers_list2 = ['', '', 'crust', 'crust', '']
colors_list = ['lightskyblue', 'tan', 'orange', 'olive', 'pink']
density_list = ['1030', '2600', '2850', '2885', '3250']

ncols = len(colors_list)
nrows = 1
h = Y / nrows
w = X / (ncols + 1)

i=ncols-1
for color, density, layers1, layers2 in zip(colors_list, density_list, layers_list1, layers_list2):
    col = i // nrows
    row = i % nrows
    x = X - (col*w) - w
    
    yi_line = Y
    yf_line = Y - Y*0.15
    yi_text1 = Y - Y*0.2
    yi_text2 = Y - Y*0.32
    yi_text3 = Y - Y*0.08
    
    i-=1
    poly = Polygon(np.array([[x, x+w*0.75, x+w*0.75, x], [yi_line, yi_line, yf_line, yf_line]]).T)
    tmpx = [x for x in poly.x]
    tmpx.append(poly.x[0])
    tmpy = [y for y in poly.y]
    tmpy.append(poly.y[0])
    ax5.plot(tmpx, tmpy, linestyle='-', color='k', linewidth=1)
    ax5.fill(tmpx, tmpy, color=color)
    ax5.text(x+w*0.375, yi_text1, layers1, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax5.text(x+w*0.375, yi_text2, layers2, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax5.text(x+w*0.375, yi_text3, density, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='center')

plt.tight_layout()    
    
#mpl.savefig('../manuscript/figures/B-model-rifted-margin-true-model-grafics.png', dpi='figure', bbox_inches='tight')
plt.show()      

In [ ]:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(12,7))

import matplotlib.gridspec as gridspec
heights = [8, 1]
gs = gridspec.GridSpec(2, 1, height_ratios=heights)
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

ax1.axhline(y=0.0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=1)
aux = yc <= COT
for (pwi) in (polygons_water):
    tmpx = [x for x in pwi.x]
    tmpx.append(pwi.x[0])
    tmpy = [y for y in pwi.y]
    tmpy.append(pwi.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='lightskyblue')
for (ps0i) in (polygons_sediments0):
    tmpx = [x for x in ps0i.x]
    tmpx.append(ps0i.x[0])
    tmpy = [y for y in ps0i.y]
    tmpy.append(ps0i.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='tan')
for (ps1i) in (polygons_sediments1):
    tmpx = [x for x in ps1i.x]
    tmpx.append(ps1i.x[0])
    tmpy = [y for y in ps1i.y]
    tmpy.append(ps1i.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='tan')
for (pci) in (polygons_crust[:len(yc[aux])]):
    tmpx = [x for x in pci.x]
    tmpx.append(pci.x[0])
    tmpy = [y for y in pci.y]
    tmpy.append(pci.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='orange')
for (pcoi) in (polygons_crust[len(yc[aux]):n]):
    tmpx = [x for x in pcoi.x]
    tmpx.append(pcoi.x[0])
    tmpy = [y for y in pcoi.y]
    tmpy.append(pcoi.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='olive')
for (pmi) in (polygons_mantle):
    tmpx = [x for x in pmi.x]
    tmpx.append(pmi.x[0])
    tmpy = [y for y in pmi.y]
    tmpy.append(pmi.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='pink')

#ax1.axhline(y=S0, xmin=ymin, xmax=ymax, color='w', linestyle='--', linewidth=3)
#ax1.axvline(x=COT, ymin=ybottom, ymax=ytop, linestyle=':', color='w', linewidth=3, label='COT')
ax1.plot(yc, tw, '-k', linewidth=3)
#ax1.plot(yc, tos, '-k', linewidth=3)
ax1.plot(yc, basement, '-k', linewidth=3, label='true surfaces')
ax1.plot(yc, moho, '-k', linewidth=3)
ax1.axhline(y=S0+dS0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=3)
#ax1.set_ylim((S0+dS0), zmin)
ax1.set_ylim((57500.0), zmin)
ax1.set_xlim(ymin, ymax)
ax1.set_xlabel('y (km)', fontsize=16)
ax1.set_ylabel('z (km)', fontsize=16)
ax1.set_xticklabels(['%g'% (0.001*l) for l in ax4.get_xticks()], fontsize=14)
ax1.set_yticklabels(['%g'% (0.001*l) for l in ax4.get_yticks()], fontsize=14)
#ax1.legend(loc='lower right', fontsize=14, facecolor='silver')    

X, Y = fig.get_dpi()*fig.get_size_inches()
plt.title('Density (kg/m$^{3}$)', fontsize=18)
ax2.axis('off')

layers_list1 = ['water', 'sediment', 'continental', 'oceanic', 'mantle']
layers_list2 = ['', '', 'crust', 'crust', '']
colors_list = ['lightskyblue', 'tan', 'orange', 'olive', 'pink']
density_list = ['1030', '2600', '2850', '2885', '3250']
ncols = len(colors_list)
nrows = 1
h = Y / nrows
w = X / (ncols + 1)

i=ncols-1
for color, density, layers1, layers2 in zip(colors_list, density_list, layers_list1, layers_list2):
    col = i // nrows
    row = i % nrows
    x = X - (col*w) - w
    
    yi_line = Y
    yf_line = Y - Y*0.15
    yi_text1 = Y - Y*0.2
    yi_text2 = Y - Y*0.27
    yi_text3 = Y - Y*0.08
    
    i-=1
    poly = Polygon(np.array([[x, x+w*0.75, x+w*0.75, x], [yi_line, yi_line, yf_line, yf_line]]).T)
    tmpx = [x for x in poly.x]
    tmpx.append(poly.x[0])
    tmpy = [y for y in poly.y]
    tmpy.append(poly.y[0])
    ax2.plot(tmpx, tmpy, linestyle='-', color='k', linewidth=1)
    ax2.fill(tmpx, tmpy, color=color)
    ax2.text(x+w*0.375, yi_text1, layers1, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax2.text(x+w*0.375, yi_text2, layers2, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax2.text(x+w*0.375, yi_text3, density, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='center')

plt.tight_layout()
    
#mpl.savefig('../manuscript/figures/B-model-rifted-margin-true-model.png', dpi='figure', bbox_inches='tight')
plt.show()      